In [1]:
import folium
from folium.plugins import HeatMap

import pickle

with open("one_obj_to_rule_them_all.pkl", "rb") as p:
    obj = pickle.load(p)
    
DELTA_COL_NAME = "SnowEx - Snotel Delta Depth (in)"

heat_map_sntl = "622:CO:SNTL"
gdf = obj["data"].sort_values("distance")
gdf = gdf[gdf["awdb_id"] == heat_map_sntl].astype({"date": str})
gdf[DELTA_COL_NAME] = gdf["depth_snowex"] - gdf["depth_awdb"]
gdf_pnt_data = gdf[gdf["instrument"] != "lidar"]

m_heat_map = gdf.drop_duplicates(subset=["distance"]).explore(
    column=DELTA_COL_NAME,
    scheme="naturalbreaks",
    legend=True,
    k=5,
    legend_kwds=dict(colorbar=False),
    name="Snow Pits",
)
sntl_meta = obj["meta"]["awdb"][obj["meta"]["awdb"]["site_id"] == heat_map_sntl]
sntl_geo = sntl_meta["geometry"].values[0]
sntl_name = sntl_meta["site_name"].values[0]
folium.Marker(
    [sntl_geo.y, sntl_geo.x],
    popup=f"<b>{sntl_name}</b> ({heat_map_sntl})",
    tooltip=f"{sntl_name}"
).add_to(m_heat_map)
heat_map_data = []
for idx, row in gdf.iterrows():
    if row["instrument"] == "lidar":
        heat_pnt = [row["geometry"].y, row["geometry"].x, row[DELTA_COL_NAME]]
        heat_map_data.append(heat_pnt)
if False:#heat_map_data:
    heat_map_lyr = HeatMap(
        heat_map_data, 
        name="Heat Map", 
        min_opacity=0.5,
        max_zoom=25, 
        radius=25, 
        blur=15, 
        gradient=None, 
        overlay=True, 
        control=True, 
        show=True
    ).add_to(m_heat_map)
folium.map.LayerControl().add_to(m_heat_map)
m_heat_map